In [31]:
import re as regex, string
from typing import List
from enchant.checker import SpellChecker
import numba
from tqdm import tnrange, tqdm_notebook
from time import sleep
import nltk

In [32]:
import logging
import sys
logging.basicConfig(format='[%(funcName)s]: %(message)s',
                     level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger("logger")
logger.setLevel(logging.DEBUG)
logger.info('Hello world!')
logger.debug("Hello deubg")

[<module>]: Hello world!
[<module>]: Hello deubg


## 读取文件

In [33]:
filen = 'data/5/testData.txt'
fdata = list(open(filen))
fdata = fdata[0:1000]

## 分词

In [34]:
from nltk.tokenize import word_tokenize
def split_words(fdata: List[List])->List[List]:
    ret = []
    for row in tqdm_notebook(fdata):
        words = word_tokenize(row)
        ret.append(words)
    return ret

In [35]:
passage = split_words(fdata)

In [36]:
print(sum(len(row) for row in passage))

387969


## 去除标点、特殊符号、HTML标签等非英文内容

In [ ]:
def remove_punc(s):
    # 使用正则表达式去除标点和特殊符号
    regex_cleaner = regex.compile("[^a-zA-Z]")
    return regex_cleaner.sub(' ', s)

In [ ]:
#执行
fdata = [remove_punc(row) for row in fdata]
words = [str.split(row) for row in fdata]

In [37]:
def remove_punc(row_of_words: List[List])->List[List]:
    ret = []
    for row in tqdm_notebook(row_of_words):
        words = [word for word in row if word.isalpha()]
        ret.append(words)
    return ret

In [38]:
passage = remove_punc(passage)
print(passage[1])


['its', 'as', 'though', 'an', 'alien', 'made', 'this', 'movie', 'its', 'really', 'disturbing', 'i', 'loved', 'it', 'some', 'great', 'directing', 'very', 'great', 'acting', 'this', 'film', 'is', 'practically', 'theatrical', 'a', 'great', 'script', 'a', 'great', 'plot', 'lots', 'of', 'symbolism', 'some', 'really', 'heavy', 'humour', 'very', 'english', 'and', 'pompous', 'and', 'rude', 'this', 'film', 'is', 'drenched', 'in', 'scarlet', 'sin', 'after', 'you', 'watch', 'it', 'you', 'feel', 'as', 'though', 'your', 'soul', 'and', 'has', 'turned', 'all', 'murky', 'but', 'it', 'was', 'still', 'very', 'much', 'worth', 'watching', 'a', 'good', 'laugh', 'for', 'those', 'special', 'few', 'who', 'are', 'open', 'minded', 'enough', 'to', 'enjoy', 'this', 'really', 'true', 'masterpiece', 'i', 'would', 'not', 'know', 'for', 'sure', 'which', 'genre', 'to', 'place', 'it', 'a', 'cult', 'classic', 'maybe', 'this', 'is', 'basically', 'the', 'story', 'of', 'the', 'owner', 'of', 'a', 'french', 'restaurant', 'h

In [40]:
print(sum(len(row) for row in passage))

331751


## 去除停用词

In [ ]:
from nltk.corpus import stopwords
def remove_stop_words(passage):
    stop_words = set(stopwords.words('english'))
    stop_words.add('us')
    passage = [list(filter(lambda w: w.lower() not in stop_words, row)) for row in passage]
    return passage


In [ ]:
passage = remove_stop_words(passage)
print(sum(len(row) for row in passage))
a = [set(row) for row in words]
ret = set()
for row in a:
    ret = ret | set(row)

## 拼写检查

In [ ]:
def sentence_spell_check(fdata)-> List[List]:
    sentence_corrected = []
    chkr = SpellChecker("en_US")
    cnt = 0
    for row in tqdm_notebook(fdata):
        chkr.set_text(row)
        for err in chkr:
            cnt += 1
            err.replace("")
        sentence_corrected.append(err.get_text())
    logging.info("There are {} errors ".format(cnt))
    return sentence_corrected

In [ ]:
def words_spell_check(fdata)->List[List]:
    words_corrected = []
    chkr = SpellChecker("en_US")
    err = 0
    cnt = 0
    for row in tqdm_notebook(fdata):
        row_corrected = []
        for word in row:
            if word.istitle():
                continue
            ret = chkr.check(word)
            cnt+=1
            if ret is False:
                err += 1
                suggest = chkr.suggest(word)
                suggest2 = autocorrect.spell(word)
                if len(suggest) == 0:
                    continue
                else:
                    print(word, suggest[0], suggest2)
            else:
                row_corrected.append(word)
        words_corrected.append(row_corrected)
    logging.info("There are {} errors in {} words, error rate : {}".format(err, cnt, err/cnt))
    return words_corrected


In [ ]:
print("There are ", sum(len(row) for row in words)," words in total")
fdata = sentence_spell_check(fdata)

In [44]:
import autocorrect

def words_spell_check(fdata)->List[List]:
    ret = []
    err = 0
    cnt = 0
    for row in tqdm_notebook(fdata):
        corrected_row = []
        for word in row:
            suggest = autocorrect.spell(word)
            if word != suggest:
                err += 1
            cnt += 1
            corrected_row.append(word)
        ret.append(corrected_row)
    logging.info("There are {} errors in {} words, error rate : {}".format(err, cnt, err/cnt))
    return ret

import os, time, random
def correct_words(passage):
    print('Run task (%s)...' % (os.getpid()))
    ret = [[autocorrect.spell(word) for word in row] for row in passage]
    return ret

In [45]:
from multiprocessing import Pool
from typing import List, NoReturn, Callable
def list_multiprocess(lst: List, func: Callable[[List],List], n: int)-> List:
    if len(lst) < n:
        return func(lst)
    p = Pool(n)
    lists = []
    psize = int(len(lst) / n)
    for i in range(n - 1):
        lists.append(lst[i * psize: (i+1) * psize])
    lists.append(lst[(n-1) * psize:])
    ret = []
    for i in range(n):
        ret.append(p.apply_async(func, args=(lists[i],)))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    last = []
    for i in ret:
        last += i.get()
    print('All subprocesses done.')
    return last

In [46]:
logger.setLevel(logging.INFO)
passage = list_multiprocess(passage, words_spell_check, 6)

Waiting for all subprocesses done...

[words_spell_check]: There are 1280 errors in 53425 words, error rate : 0.02395882077678989

[words_spell_check]: There are 1310 errors in 59600 words, error rate : 0.02197986577181208

[words_spell_check]: There are 1323 errors in 52671 words, error rate : 0.025118186478327732

[words_spell_check]: There are 1355 errors in 53657 words, error rate : 0.02525299588124569

[words_spell_check]: There are 1195 errors in 51877 words, error rate : 0.023035256472039634

[words_spell_check]: There are 1386 errors in 60521 words, error rate : 0.022901141752449564
All subprocesses done.


In [47]:
passage[0]

['playwright',
 'james',
 'barrie',
 'johnny',
 'depp',
 'finds',
 'his',
 'career',
 'at',
 'a',
 'crossroads',
 'when',
 'his',
 'latest',
 'play',
 'flops',
 'and',
 'doubters',
 'question',
 'his',
 'future',
 'then',
 'by',
 'chance',
 'he',
 'meets',
 'a',
 'widow',
 'kate',
 'winslet',
 'and',
 'her',
 'four',
 'adventurous',
 'boys',
 'together',
 'they',
 'form',
 'a',
 'friendship',
 'that',
 'ignites',
 'the',
 'imagination',
 'needed',
 'to',
 'produce',
 'barrie',
 'greatest',
 'work',
 'an',
 'enchanting',
 'treat',
 'with',
 'an',
 'acclaimed',
 'cast',
 'of',
 'stars',
 'finding',
 'neverland',
 'has',
 'been',
 'hailed',
 'as',
 'one',
 'of',
 'the',
 'year',
 'best',
 'motion',
 'pictures',
 'according',
 'to',
 'the',
 'film',
 'official',
 'synopsis',
 'this',
 'biography',
 'of',
 'the',
 'famed',
 'peter',
 'pan',
 'author',
 'is',
 'noted',
 'to',
 'have',
 'been',
 'inspired',
 'by',
 'true',
 'events',
 'during',
 'the',
 'opening',
 'but',
 'it',
 'plays',
 'l

In [ ]:
chkr = SpellChecker("en_US")
chkr.set_text("I am a happi happi boy")
for err in chkr:
    print(str(err.word))
    err.replace("")
chkr.get_text()

## 词性标注

In [ ]:
def get_word_properties(row):
    ret = []
    for row in tqdm_notebook(words):
        word_properties = nltk.pos_tag(row)
        ret.append(word_properties)
    return ret
word_class =get_word_properties(corrected)
print(word_class[0])


In [48]:
from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

## 词形还原

In [ ]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
print(wnl.lemmatize('countries'))  

In [ ]:
import nltk  
sent1='The cat is walking in all countries.'  
sent2='A dog was running across the kitchen.'  
tokens_1=nltk.word_tokenize(sent2)  
print (tokens_1)
stemmer = nltk.stem.PorterStemmer()
print(wnl.lemmatize('was'))  
stem_0 = [wnl.lemmatize(t) for t in tokens_1]  
print(stem_0) 


In [49]:
from pattern.en import lemma
lemma('countries')

RuntimeError: generator raised StopIteration

In [ ]:
def lemma_words(passage):
    ret = []
    for row in passage:
        ret.append([lemma(word) for word in row])
    return ret
passage_lemma = lemma_words(passage)

In [1]:
from nltk.stem import WordNetLemmatizer
def lemma_passage(passage):
    ret = list(range(len(passage)))
    lemmatizer = WordNetLemmatizer()
    for i, row in tqdm_notebook(enumerate(passage)):
        nrow = []
        for w, pos in nltk.pos_tag(row):
            wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
            nrow.append(lemmatizer.lemmatize(w.lower(), pos=wordnet_pos))
        ret[i] = nrow
    return ret

In [51]:
passage_lemma = lemma_passage(passage)

In [52]:
for i in range(100):
    for j in range(len(passage_lemma[i])):
        if passage_lemma[i][j] != passage[i][j]:
            print(passage_lemma[i][j], passage[i][j])

find finds
crossroad crossroads
late latest
flop flops
doubter doubters
meet meets
boy boys
ignite ignites
need needed
great greatest
enchant enchanting
star stars
find finding
have has
be been
hail hailed
a as
picture pictures
accord according
be is
note noted
be been
inspire inspired
event events
play plays
sentimentalize sentimentalized
show shows
a as
be is
do does
find finding
be is
find finding
it its
a as
make made
it its
disturb disturbing
love loved
act acting
be is
lot lots
be is
drench drenched
a as
have has
turn turned
be was
watch watching
be are
mind minded
be is
be is
be being
movie movies
be is
movie movies
do does
make makes
be is
smoke smoking
bring brings
be was
place placed
do does
be was
be were
kid kids
be is
time times
see seen
occasion occasions
get gets
shield shields
typify typifies
make makes
late later
film films
be is
mistake mistakes
age aging
danger dangers
become becoming
feel feels
a as
have had
continue continued
film films
age aging
cite cited
a as
be

be is
like likes
robert roberts
setting settings
a as
go going
it its
meaning meanings
message messages
be was
scene scenes
be is
miss missed
turn turns
do does
be is
utter uttered
go going
be is
year years
have has
result resulted
circumstance circumstances
have has
hire hired
be is
be is
use uses
introduce introducing
utter uttering
incriminate incriminating
remark remarks
be is
reveal revealed
a as
steal stolen
break breaks
act acts
confine confined
compartment compartments
need needs
reach reaches
it its
discharge discharges
it its
passenger passengers
brain brains
be is
know known
break breaks
shade shades
style styled
be is
seem seems
be was
establish established
a as
be been
recognize recognized
member members
include including
be was
intrigue intriguing
entertain entertained
find found
film films
fare fared
be was
keep keeping
piece pieces
do done
recommend recommended
enjoy enjoyed
photograph photographed
feature features
performance performances
be is
rivet riveting
watch wat

be is
be was
script scripted
year years
have had
write written
remember remembers
have has
create creating
nudist nudists
be is
have has
win won
know known
antic antics
a as
be is
assign assigned
le less
be is
have has
be been
suffer suffering
year years
have has
be been
write writing
novel novels
end ends
sniff sniffs
become becomes
convince convinced
physic physics
have has
be been
kidnap kidnapped
replace replaced
be is
leave left
start starts
introduction introductions
viewer viewers
pay pays
get gets
piece pieces
be is
prove proves
audience audiences
character characters
be been
construct constructed
be is
winner winners
steal stealing
idea ideas
be are
moment moments
look looking
be am
laugh laughed
be is
seller sellers
be is
miss missing
object objects
be is
include including
iron irons
be is
exceed exceeds
it its
be being
telegraph telegraphed
it its
be is
have had
laugh laughed
good better
comedy comedies
like liked
good better
it its
hear heard
thing things
check checked
be w

aunt aunts
keep kept
girl girls
be was
do done
lot lots
twist twists
turn turns
point pointing
finger fingers
go goes
be was
emotion emotions
run running
a as
see seeing
overwhelm overwhelming
play played
find finding
beat beating
do did
happen happened
be is
purpose purposes
emotion emotions
do did
be was
seek sought
be being
think thought
use using
tool tools
a as
shoot shooting
be was
let letting
be was
do does
do done
have has
be been
release released
want wanting
move moves
start starts
have having
encounter encounters
thing things
get getting
seem seems
be is
drown drowned
wed wedding
guy guys
be are
harpy harpies
come comes
suck sucks
have has
rat rated
situation situations
hear heard
want wanted
know knew
die died
film filming
have had
finish finished
be was
headline headlining
assume assumed
be was
have had
be was
murder murdered
sky skies
work works
a as
invent inventing
variety varieties
pie pies
hop hopes
have has
be been
save saving
hide hidden
plan plans
find finds
be is


movie movies
be is
movie movies
be is
setting settings
be are
scene scenes
be are
touch touching
be is
surprise surprised
have has
attract attracted
audience audiences
be is
have has
character characters
costume costumes
be are
film films
watch watched
be is
have has
be been
call called
elaborate elaborates
be is
be is
it its
borrow borrows
it its
it its
hero heroes
be is
age aging
be is
effect effects
lurch lurches
hall halls
mirror mirrors
be is
pass passing
situation situations
make making
value values
be is
ideal ideals
be are
reveal revealed
a as
become becomes
element elements
space spaces
be are
mock mocked
reverse reversed
whiz whizzing
be is
cream creamed
mix mixing
write wrote
direct directed
character characters
be are
depict depicts
elia elias
be is
portray portraying
be is
a as
force forces
concern concerning
be is
smile smiling
treat treating
u us
talk talking
monkey monkeys
a as
play plays
remain remains
abandon abandoned
portray portrays
a as
u us
believe believing
have

In [ ]:
passage = remove_stop_words(passage_lemma)
print(sum(len(row) for row in passage))
print(passage_lemma[0])

In [ ]:
from nltk.corpus import stopwords
def remove_stop_words(passage):
    stop_words = set(stopwords.words('english'))
    stop_words.add('us')
    passage = [list(filter(lambda w: w.lower() not in stop_words, row)) for row in passage]
    return passage


print(sum(len(row) for row in passage))

passage_try = remove_stop_words(passage)

print(sum(len(row) for row in passage_try))
print(passage[0])
print(passage_try[0])
a = [set(row) for row in passage]
ret = set()
for row in a:
    ret = ret | set(row)
print(len(ret))

In [73]:
import csv

with open("output.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(passage)

In [72]:
with open("data/5/testData_processed_compressed.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(passage_try)

In [78]:
# 引入 word2vec
from gensim.models import word2vec
 
# 引入日志配置
import logging
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
 
# 构建模型
model = word2vec.Word2Vec(passage, min_count=1)
 
# 进行相关性比较
model.similarity('table','dog')

[scan_vocab]: collecting all words and their counts
[_scan_vocab]: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[scan_vocab]: collected 20467 word types from a corpus of 331751 raw words and 1000 sentences
[prepare_vocab]: Loading a fresh vocabulary
[prepare_vocab]: effective_min_count=1 retains 20467 unique words (100% of original 20467, drops 0)
[prepare_vocab]: effective_min_count=1 leaves 331751 word corpus (100% of original 331751, drops 0)
[prepare_vocab]: deleting the raw counts dictionary of 20467 items
[prepare_vocab]: sample=0.001 downsamples 44 most-common words
[prepare_vocab]: downsampling leaves estimated 248753 word corpus (75.0% of prior 331751)
[estimate_memory]: estimated required memory for 20467 words and 100 dimensions: 26607100 bytes
[reset_weights]: resetting layer weights
[_check_training_sanity]: training model with 3 workers on 20467 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
[_log_epoch_progress]: worker

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  del sys.path[0]
/usr/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9928684

In [81]:
from gensim.test.utils import common_texts

[add_documents]: adding document #0 to Dictionary(0 unique tokens: [])
[add_documents]: built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [79]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [82]:
print(common_texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [88]:
documents = list(range(len(common_texts)))
for i, doc in enumerate(common_texts):
    documents[i] = TaggedDocument(doc, [i])

In [90]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=12)

[__init__]: consider setting layer size to a multiple of 4 for greater performance
[scan_vocab]: collecting all words and their counts
[_scan_vocab]: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
[scan_vocab]: collected 12 word types and 9 unique tags from a corpus of 9 examples and 29 words
[prepare_vocab]: Loading a fresh vocabulary
[prepare_vocab]: effective_min_count=1 retains 12 unique words (100% of original 12, drops 0)
[prepare_vocab]: effective_min_count=1 leaves 29 word corpus (100% of original 29, drops 0)
[prepare_vocab]: deleting the raw counts dictionary of 12 items
[prepare_vocab]: sample=0.001 downsamples 12 most-common words
[prepare_vocab]: downsampling leaves estimated 3 word corpus (12.1% of prior 29)
[estimate_memory]: estimated required memory for 12 words and 5 dimensions: 6660 bytes
[reset_weights]: resetting layer weights
[_check_training_sanity]: training model with 4 workers on 12 vocabulary and 5 features, using sg=0 hs=0 sample=0.00

In [91]:
from gensim.test.utils import get_tmpfile
fname = get_tmpfile("my_doc2vec_model")
model.save(fname)

[_smart_save]: saving Doc2Vec object under /tmp/my_doc2vec_model, separately None
[_smart_save]: saved /tmp/my_doc2vec_model


In [92]:
vector = model.infer_vector(["system", "response"])

In [93]:
vector

array([-0.05454625, -0.03885484, -0.03811057, -0.0459829 , -0.01424503],
      dtype=float32)